# Dealing with imbalanced datasets, combining oversampling with VAE and undersampling to improve model recognition over all classes.  

Import packages, classifiers and etc.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from sklearn.metrics import confusion_matrix, make_scorer
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from collections import Counter

from imblearn.over_sampling import SMOTE, RandomOverSampler

Import VAEOversampler.

In [2]:
from VAEOversampler import VAEOversampler

2023-07-11 12:19:50.228457: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-11 12:19:50.228494: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading data  
You can load some dataset from Imbalanced Learn list (https://imbalanced-learn.org/stable/datasets/index.html) or use your own data.  


In [3]:
from imblearn.datasets import fetch_datasets

dset_name = 'ozone_level'
dset = fetch_datasets()[dset_name]

X, y = StandardScaler().fit_transform(dset.data), dset.target

In [4]:
X

array([[-0.71397271,  0.02029905,  0.44761066, ...,  2.96740577,
        -1.38030021, -0.28234978],
       [ 0.66903584,  0.97486511,  1.06809445, ...,  1.97728853,
        -1.38030021, -0.28234978],
       [ 0.73818626,  0.70213195,  0.58549595, ...,  1.25720327,
        -1.00522613, -0.28234978],
       ...,
       [-0.71397271, -0.66153385, -0.37970107, ...,  1.97728853,
        -0.88020144, -0.28234978],
       [-0.36822057, -0.59335056, -0.17287314, ...,  1.43722459,
        -0.75517675, -0.24437611],
       [-0.22991972, -0.3206174 ,  0.0339548 , ...,  0.9871713 ,
        -0.63015206, -0.28234978]])

In [5]:
y[y == -1] = 0

We split data into train and test partitions.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

This is a simple function to undersample freely.  

In [7]:
# RUS

def RUS(X_res, y_res, frac=1, minority_class_id=1, random_state=42):
    X_res = pd.DataFrame(X_res)
    X_res['Class'] = y_res

    X_neg = X_res[y_res != minority_class_id].sample(frac=frac, random_state=random_state)
    X_pos = X_res[y_res == minority_class_id].sample(frac=1, random_state=random_state)

    X_rus = pd.concat([X_neg, X_pos], ignore_index=True)

    X_eq = X_rus.drop('Class', axis=1)
    y_eq = X_rus['Class']

    return X_eq, y_eq

In [8]:
def train_val(X, y, Xt, yt, random_state=42):
    classifiers = {
        "CatBoostClassifier": CatBoostClassifier(verbose=False, random_seed=random_state),
        "LGBMClassifier": LGBMClassifier(random_state=random_state),
        "XGBClassifier": XGBClassifier(random_state=random_state),
        "BaggingClassifier": BaggingClassifier(random_state=random_state),
        "RandomForestClassifier": RandomForestClassifier(random_state=random_state),
    }
    scores = []
    predictions = []
    for key, classifier in classifiers.items():
        print('_' * 50)
        name = key
        classifier.fit(X, y)
        print("Classifier: ", name)
        y_pred = classifier.predict(Xt)
        cm = confusion_matrix(yt, y_pred)
        print(cm)
        print('')
        predictions.append(y_pred)
        tn = cm[0,0]
        fp = cm[0,1]
        fn = cm[1,0]
        tp = cm[1,1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)
        scores.append(tnr * tpr)
        print('TNR:', round(tnr, 5))
        print('TPR:', round(tpr, 5))
        print('TNRxTPR:', round(tnr * tpr, 5))
        print('G-mean:', round(np.sqrt(tnr * tpr), 5))

    print('_' * 50)
    print('Ensemble predictions (majority voting):')
    predictions = np.sum(predictions, axis=0)
    predictions[predictions < 3] = 0
    predictions[predictions >= 3] = 1

    cm = confusion_matrix(yt, predictions)
    print(cm)
    tn = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    tp = cm[1,1]
    tnr = tn / (tn + fp)
    tpr = tp / (tp + fn)
    print('')
    print('TNR:', round(tnr, 5))
    print('TPR:', round(tpr, 5))
    print('TNRxTPR:', round(tnr * tpr, 5))
    print('G-mean:', round(np.sqrt(tnr * tpr), 5))


## Without resampling (base line) 
Which is starting score?


In [9]:
print('Original dataset shape %s' % Counter(y))
print('Ratio->', round(Counter(y)[0]/Counter(y)[1], 1), ': 1')

Original dataset shape Counter({0: 2463, 1: 73})
Ratio-> 33.7 : 1


In [10]:
train_val(X_train, y_train, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[491   2]
 [ 15   0]]

TNR: 0.99594
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  LGBMClassifier
[[490   3]
 [ 14   1]]

TNR: 0.99391
TPR: 0.06667
TNRxTPR: 0.06626
G-mean: 0.25741
__________________________________________________
Classifier:  XGBClassifier
[[491   2]
 [ 15   0]]

TNR: 0.99594
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  BaggingClassifier
[[493   0]
 [ 15   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  RandomForestClassifier
[[492   1]
 [ 15   0]]

TNR: 0.99797
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[491   2]
 [ 15   0]]

TNR: 0.99594
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0


## Ratio 1:1  
Let's see classifiers scores when dataset is balanced.  


In [11]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f62c71aadc0>
Traceback (most recent call last):
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


In [12]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 1970, 1: 1970})
Ratio->  1 : 1.0


In [13]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[481  12]
 [ 13   2]]

TNR: 0.97566
TPR: 0.13333
TNRxTPR: 0.13009
G-mean: 0.36068
__________________________________________________
Classifier:  LGBMClassifier
[[485   8]
 [ 13   2]]

TNR: 0.98377
TPR: 0.13333
TNRxTPR: 0.13117
G-mean: 0.36217
__________________________________________________
Classifier:  XGBClassifier
[[482  11]
 [ 12   3]]

TNR: 0.97769
TPR: 0.2
TNRxTPR: 0.19554
G-mean: 0.4422
__________________________________________________
Classifier:  BaggingClassifier
[[483  10]
 [ 15   0]]

TNR: 0.97972
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  RandomForestClassifier
[[488   5]
 [ 13   2]]

TNR: 0.98986
TPR: 0.13333
TNRxTPR: 0.13198
G-mean: 0.36329
__________________________________________________
Ensemble predictions (majority voting):
[[486   7]
 [ 13   2]]

TNR: 0.9858
TPR: 0.13333
TNRxTPR: 0.13144
G-mean: 0.36255


In [14]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [15]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 1970, 1: 1970})
Ratio->  1 : 1.0


In [16]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[487   6]
 [ 14   1]]

TNR: 0.98783
TPR: 0.06667
TNRxTPR: 0.06586
G-mean: 0.25662
__________________________________________________
Classifier:  LGBMClassifier
[[483  10]
 [ 13   2]]

TNR: 0.97972
TPR: 0.13333
TNRxTPR: 0.13063
G-mean: 0.36143
__________________________________________________
Classifier:  XGBClassifier
[[485   8]
 [ 14   1]]

TNR: 0.98377
TPR: 0.06667
TNRxTPR: 0.06558
G-mean: 0.2561
__________________________________________________
Classifier:  BaggingClassifier
[[489   4]
 [ 14   1]]

TNR: 0.99189
TPR: 0.06667
TNRxTPR: 0.06613
G-mean: 0.25715
__________________________________________________
Classifier:  RandomForestClassifier
[[492   1]
 [ 15   0]]

TNR: 0.99797
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[489   4]
 [ 14   1]]

TNR: 0.99189
TPR: 0.06667
TNRxTPR: 0.06613
G-mean: 0.25715


In [17]:
# VAEOversampler

vae_sampler = VAEOversampler(epochs=500,
                              intermediate_dim=512,
                              batch_size=64,
                              random_state=42,
                              verbose=False)
X_res, y_res = vae_sampler.fit_resample(X_train, y_train)

2023-07-11 12:21:42.253691: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-11 12:21:42.253737: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-11 12:21:42.253772: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a-Modern-15-A5M): /proc/driver/nvidia/version does not exist
2023-07-11 12:21:42.254733: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0.0: 1970, 1.0: 1970})
Ratio->  1 : 1.0


In [19]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[491   2]
 [ 15   0]]

TNR: 0.99594
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  LGBMClassifier
[[490   3]
 [ 15   0]]

TNR: 0.99391
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  XGBClassifier
[[490   3]
 [ 15   0]]

TNR: 0.99391
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  BaggingClassifier
[[492   1]
 [ 15   0]]

TNR: 0.99797
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Classifier:  RandomForestClassifier
[[493   0]
 [ 15   0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[492   1]
 [ 15   0]]

TNR: 0.99797
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0


## Under/Oversampling combination  
Now we can tuning the number of instances for each class to optimize metric.  


In [27]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f62086ca0d0>
Traceback (most recent call last):
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/a/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


In [30]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.05)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 1970, 0: 98})
Ratio->  1 : 20.1


In [31]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[352 141]
 [  5  10]]

TNR: 0.714
TPR: 0.66667
TNRxTPR: 0.476
G-mean: 0.68993
__________________________________________________
Classifier:  LGBMClassifier
[[384 109]
 [  4  11]]

TNR: 0.7789
TPR: 0.73333
TNRxTPR: 0.5712
G-mean: 0.75578
__________________________________________________
Classifier:  XGBClassifier
[[374 119]
 [  4  11]]

TNR: 0.75862
TPR: 0.73333
TNRxTPR: 0.55632
G-mean: 0.74587
__________________________________________________
Classifier:  BaggingClassifier
[[361 132]
 [  8   7]]

TNR: 0.73225
TPR: 0.46667
TNRxTPR: 0.34172
G-mean: 0.58457
__________________________________________________
Classifier:  RandomForestClassifier
[[366 127]
 [  5  10]]

TNR: 0.74239
TPR: 0.66667
TNRxTPR: 0.49493
G-mean: 0.70351
__________________________________________________
Ensemble predictions (majority voting):
[[369 124]
 [  4  11]]

TNR: 0.74848
TPR: 0.73333
TNRxTPR: 0.54888
G-mean: 0.74087


In [32]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [43]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.05)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 1970, 0: 98})
Ratio->  1 : 20.1


In [44]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[355 138]
 [  4  11]]

TNR: 0.72008
TPR: 0.73333
TNRxTPR: 0.52806
G-mean: 0.72668
__________________________________________________
Classifier:  LGBMClassifier
[[391 102]
 [  5  10]]

TNR: 0.7931
TPR: 0.66667
TNRxTPR: 0.52874
G-mean: 0.72714
__________________________________________________
Classifier:  XGBClassifier
[[379 114]
 [  5  10]]

TNR: 0.76876
TPR: 0.66667
TNRxTPR: 0.51251
G-mean: 0.7159
__________________________________________________
Classifier:  BaggingClassifier
[[362 131]
 [  7   8]]

TNR: 0.73428
TPR: 0.53333
TNRxTPR: 0.39162
G-mean: 0.62579
__________________________________________________
Classifier:  RandomForestClassifier
[[388 105]
 [  5  10]]

TNR: 0.78702
TPR: 0.66667
TNRxTPR: 0.52468
G-mean: 0.72435
__________________________________________________
Ensemble predictions (majority voting):
[[379 114]
 [  4  11]]

TNR: 0.76876
TPR: 0.73333
TNRxTPR: 0.56376
G-mean: 0.75084


In [58]:
# VAEOversampler

X_res, y_res = vae_sampler.resample(X_train, y_train, sampling_strategy=1.5)

In [61]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.04)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1.0: 2926, 0.0: 79})
Ratio->  1 : 37.0


In [62]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[381 112]
 [  5  10]]

TNR: 0.77282
TPR: 0.66667
TNRxTPR: 0.51521
G-mean: 0.71778
__________________________________________________
Classifier:  LGBMClassifier
[[410  83]
 [  4  11]]

TNR: 0.83164
TPR: 0.73333
TNRxTPR: 0.60987
G-mean: 0.78094
__________________________________________________
Classifier:  XGBClassifier
[[411  82]
 [  4  11]]

TNR: 0.83367
TPR: 0.73333
TNRxTPR: 0.61136
G-mean: 0.78189
__________________________________________________
Classifier:  BaggingClassifier
[[395  98]
 [  5  10]]

TNR: 0.80122
TPR: 0.66667
TNRxTPR: 0.53414
G-mean: 0.73085
__________________________________________________
Classifier:  RandomForestClassifier
[[397  96]
 [  6   9]]

TNR: 0.80527
TPR: 0.6
TNRxTPR: 0.48316
G-mean: 0.6951
__________________________________________________
Ensemble predictions (majority voting):
[[404  89]
 [  5  10]]

TNR: 0.81947
TPR: 0.66667
TNRxTPR: 0.54632
G-mean: 0.73913


## References  

  - Classification with Imbalanced Datasets:  
    https://sci2s.ugr.es/imbalanced  
  - Computer Vision:  Models, Learning, and Inference (Simon J.D. Prince):  
    http://www.computervisionmodels.com/  
  - Oversampling with VAEs:  
    https://towardsdatascience.com/oversampling-with-vaes-e410887fe51  
